In [1]:
import sys
sys.path.append("../../")
import json
import urllib.request
from IPython.display import clear_output
import sources
from countries.list import CountryList
from data.Frame import Frame
from data.Stats import calculate_weighted_average as wa
c = CountryList()
c.load_wb()
from sources.worldbank.indicators import get_data_frame_wide as getdfw
from sources.worldbank.datatable import get_data_table

In [2]:
import bokeh
from bokeh.io import output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure, output_file, show
output_notebook()

Loading BokehJS ...

In [3]:
def estimate(years,indicator, groups, weight = None):
    data_df = get_data_table(years, indicator, groups, export_to_excel=False)    
    
    if weight is None:
        return (data_df)    
    
    weight_df = get_data_table(years, weight, groups, export_to_excel=False)
    df_ldcs_w = wa(data_df, weight_df, 'entity' )
    
    return (df_ldcs_w)

In [4]:
def line_chart(series, title):
    output_file("data.html")
    # create a new plot with a title and axis labels
    p = figure(title=title, x_axis_label='Years', y_axis_label='Percentage', height = 400, width = 900)    
    colors = ['red', 'blue', 'green', 'orange']
    j = 0
    for data in series:                
        # output to static HTML file
        # add a line renderer with legend and line thickness
        x = data[0]
        y = data[1]
        group = data[2]
        p.line(x, y, legend=group, line_width=2, color = colors[j])
        p.circle(x, y, color=colors[j], fill_color=colors[j], line_width=3)
        j = j + 1
    
    hover = HoverTool()
    hover.tooltips = """
        <div style=padding=5px>Year:@x</div>
        <div style=padding=5px>Value:@y</div>
        """
    p.add_tools(hover)
    
    return p

In [5]:
def show_weighted_average(indicator, weight, groups):
    datasets = []
    for group in groups:
        data_group = estimate(years, indicator, [[group]], weight)
        x = years
        y = []
        for i in range(3,len(data_group.get_last_row().get_array()), 3):
            y.append(data_group.get_last_row()[i].get_value())
        data_group.entities_description = group
        datasets.append([x,y, group, data_group.description])
    series = datasets
    show(line_chart(series, series[0][3]))

In [30]:
def show_group_total(indicator, groups):
    datasets = []
    for group in groups:
        data_group = get_data_table(years, indicator, [[group]])
        x = years
        y = []
        for i in range(1,len(data_group.get_total_by_column())):
            y.append(data_group.get_total_by_column()[i])
        data_group.entities_description = group
        datasets.append([x,y, group, data_group.description])
    series = datasets
    show(line_chart(series, series[0][3]))        

In [17]:
years = list(range(2001,2018))
groups_ldcs=[['LDCs']]
groups_odcs =[['Developing excluding LDCs']]
groups_sids = [['SIDS']]
groups_lldcs = [['LLDCs']]
groups_oecd = [['OECD']]
groups_mics = [['MICs']]

In [18]:
show_weighted_average('NV.AGR.TOTL.ZS', 'NY.GDP.MKTP.CD', ['LDCs', 'Developing excluding LDCs'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [19]:
# Urban population as a percentage of total population
show_weighted_average('SP.RUR.TOTL.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [20]:
# Access to safely managed sanitation services
show_weighted_average('SH.STA.SMSS.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [21]:
# Literacy rate, youth female (% of females ages 15-24)
show_weighted_average('SE.ADT.1524.LT.FE.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [35]:
# Literacy rate, youth female (% of females ages 15-24)
show_weighted_average('SP.POP.0014.TO.ZS', 'SP.POP.TOTL', ['LDCs', 'Developing excluding LDCs', 'OECD'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [40]:
years = list(range(1960,2018))
show_group_total('SP.POP.TOTL', ['LDCs'])

INFO:bokeh.io.state:Session output file 'data.html' already exists, will be overwritten.


In [28]:
data_group = get_data_table(years, 'SP.POP.TOTL', [['LDCs']])